# Alpaca

### BUSI 722: Data-Driven Finance II
### Kerry Back, Rice University

## Outline

1. Create connection
2. View account information
3. View market information
4. Make trades

Note: log on to alpaca at https://alpaca.markets/ to get key and secret key

In [43]:
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce
from alpaca.trading.requests import GetAssetsRequest
from alpaca.data import StockHistoricalDataClient
from alpaca.data.requests import StockLatestQuoteRequest
from alpaca.trading.enums import AssetClass
import pandas as pd

## 1. Create Connection

In [44]:
KEY = "PKWPSIEQ8SCBOXYJHZ0G"
SECRET_KEY = "g6IrUta5cx8yzJnIPrx2CPG2vvzjCvPqDvZgqyDM"
trading_client = TradingClient(KEY, SECRET_KEY, paper=True)

## 2. Account Information

In [45]:
account = trading_client.get_account()
account

{   'account_blocked': False,
    'account_number': 'PA3J0LUAM1VV',
    'accrued_fees': '0',
    'buying_power': '200000',
    'cash': '100000',
    'created_at': datetime.datetime(2024, 2, 11, 16, 46, 29, 12993, tzinfo=datetime.timezone.utc),
    'crypto_status': <AccountStatus.ACTIVE: 'ACTIVE'>,
    'currency': 'USD',
    'daytrade_count': 0,
    'daytrading_buying_power': '0',
    'equity': '100000',
    'id': UUID('8955151d-0998-4e09-bae8-6f8ed3979f47'),
    'initial_margin': '0',
    'last_equity': '100000',
    'last_maintenance_margin': '0',
    'long_market_value': '0',
    'maintenance_margin': '0',
    'multiplier': '2',
    'non_marginable_buying_power': '100000',
    'pattern_day_trader': False,
    'pending_transfer_in': '0',
    'pending_transfer_out': None,
    'portfolio_value': '100000',
    'regt_buying_power': '200000',
    'short_market_value': '0',
    'shorting_enabled': True,
    'sma': '0',
    'status': <AccountStatus.ACTIVE: 'ACTIVE'>,
    'trade_suspended_by_

### Examples of retrieving account details

In [58]:
print("Account balance is", account.equity)
print("Yesterday's account balance was", account.last_equity)
print("Buying power is", account.buying_power)
print("Cash balance is", account.cash)

Account balance is 100000
Yesterday's account balance was 100000
Buying power is 194019.67
Cash balance is 100000


### Open positions

In [47]:
positions = trading_client.get_all_positions()
symbol = [x.symbol for x in positions]
quantity = [float(x.qty) for x in positions]
mkt_value = [float(x.market_value) for x in positions]
cost_basis = [float(x.cost_basis) for x in positions]

positions = pd.DataFrame(
    dict(quantity=quantity, mkt_value=mkt_value, cost_basis=cost_basis),
    index=symbol
)
positions

Empty DataFrame
Columns: [quantity, mkt_value, cost_basis]
Index: []

## 3. Market Information

### Information about a particular asset

In [48]:
SYMBOL = 'SPY'

asset_info = trading_client.get_asset(SYMBOL)
if asset_info.tradable:
    print(f'We can trade {SYMBOL}.')
asset_info

We can trade SPY.


{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
    'easy_to_borrow': True,
    'exchange': <AssetExchange.ARCA: 'ARCA'>,
    'fractionable': True,
    'id': UUID('b28f4066-5c6d-479b-a2af-85dc1a8f16fb'),
    'maintenance_margin_requirement': 30.0,
    'marginable': True,
    'min_order_size': None,
    'min_trade_increment': None,
    'name': 'SPDR S&P 500 ETF Trust',
    'price_increment': None,
    'shortable': True,
    'status': <AssetStatus.ACTIVE: 'active'>,
    'symbol': 'SPY',
    'tradable': True}

### Quotes

In [49]:
lst = ['SPY','AAPL','IBM']

data_client = StockHistoricalDataClient(KEY, SECRET_KEY)
params = StockLatestQuoteRequest(symbol_or_symbols=lst)
quotes = data_client.get_stock_latest_quote(params)
quotes

{'SPY': {   'ask_exchange': 'V',
     'ask_price': 503.9,
     'ask_size': 10.0,
     'bid_exchange': 'V',
     'bid_price': 497.87,
     'bid_size': 10.0,
     'conditions': ['R'],
     'symbol': 'SPY',
     'tape': 'B',
     'timestamp': datetime.datetime(2024, 2, 9, 21, 59, 0, 970892, tzinfo=datetime.timezone.utc)},
 'IBM': {   'ask_exchange': ' ',
     'ask_price': 0.0,
     'ask_size': 0.0,
     'bid_exchange': 'V',
     'bid_price': 186.31,
     'bid_size': 1.0,
     'conditions': ['?'],
     'symbol': 'IBM',
     'tape': 'A',
     'timestamp': datetime.datetime(2024, 2, 9, 20, 59, 59, 511101, tzinfo=datetime.timezone.utc)},
 'AAPL': {   'ask_exchange': 'V',
     'ask_price': 199.0,
     'ask_size': 1.0,
     'bid_exchange': 'V',
     'bid_price': 185.0,
     'bid_size': 5.0,
     'conditions': ['R'],
     'symbol': 'AAPL',
     'tape': 'C',
     'timestamp': datetime.datetime(2024, 2, 9, 20, 59, 59, 204964, tzinfo=datetime.timezone.utc)}}

### List of all US equities (including OTC)

In [50]:
search_params = GetAssetsRequest(asset_class=AssetClass.US_EQUITY)
assets = trading_client.get_all_assets(search_params)

print(f"There are {len(assets):,} US equities.")
print("First stock in list is")
assets[0]

There are 31,628 US equities.
First stock in list is


{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
    'easy_to_borrow': False,
    'exchange': <AssetExchange.OTC: 'OTC'>,
    'fractionable': False,
    'id': UUID('574cbb96-9a53-45c8-a350-2367321145a4'),
    'maintenance_margin_requirement': 100.0,
    'marginable': False,
    'min_order_size': None,
    'min_trade_increment': None,
    'name': 'RSE ARCHIVE LLC MEMBERSHIP INT SER - 91JORDAN 1991 JORDAN GAME '
            'WORN UNIFORM',
    'price_increment': None,
    'shortable': False,
    'status': <AssetStatus.INACTIVE: 'inactive'>,
    'symbol': 'AHLFS',
    'tradable': False}

### Tradable and shortable stocks

In [51]:
assets = [x for x in assets if x.tradable]
symbols = [x.symbol for x in assets]
shortable = [x.shortable for x in assets]
shortable = pd.Series(shortable, index=symbols)

print(f"Number of tradable stocks is {len(assets):,}")
print(f"Number of shortable stocks is {shortable.sum():,}")
shortable.head()

Number of tradable stocks is 11,344
Number of shortable stocks is 4,943


ALLK      True
ADMA      True
SVIIU    False
SVIIW    False
SVIX      True
dtype: bool

### Tradable exchanges

In [52]:
set([x.exchange[:] for x in assets])

{'AMEX', 'ARCA', 'BATS', 'NASDAQ', 'NYSE', 'OTC'}

## 4. Trading

In [53]:
order = MarketOrderRequest(
    symbol="SPY",
    qty=10,
    side=OrderSide.BUY,
    time_in_force=TimeInForce.DAY
    )
_ = trading_client.submit_order(order)

In [54]:
order = MarketOrderRequest(
    symbol="AAPL",
    qty=5,
    side=OrderSide.SELL,
    time_in_force=TimeInForce.DAY
    )
_ = trading_client.submit_order(order)